In [1]:
import pulp

# 所有厨房组合 Set（1-20）
sets = {
    1:  ["T2", "W2", "L4", "C2", "O4", "D2", "S2", "V2"],
    2:  ["T2", "W1", "L1", "C4", "O4", "D2", "S4", "V2"],
    3:  ["T1", "W3", "L2", "C1", "O1", "D1", "S3", "V3"],
    4:  ["T3", "W3", "L3", "C3", "O3", "D1", "S1", "V1"],
    5:  ["T4", "W4", "L1", "C1", "O2", "D1", "S2", "V1"],
    6:  ["T2", "W2", "L2", "C4", "O4", "D2", "S3", "V4"],
    7:  ["T1", "W3", "L4", "C3", "O2", "D1", "S1", "V1"],
    8:  ["T2", "W1", "L3", "C1", "O1", "D2", "S3", "V4"],
    9:  ["T2", "W1", "L2", "C3", "O2", "D2", "S2", "V2"],
    10: ["T1", "W1", "L1", "C1", "O3", "D1", "S4", "V3"],
    11: ["T3", "W1", "L3", "C3", "O1", "D1", "S1", "V3"],
    12: ["T2", "W2", "L1", "C2", "O2", "D2", "S4", "V2"],
    13: ["T4", "W4", "L3", "C3", "O1", "D1", "S2", "V3"],
    14: ["T4", "W4", "L4", "C1", "O3", None, "S1", "V1"],
    15: ["T3", "W3", "L1", "C1", "O1", None, "S3", "V3"],
    16: ["T3", "W3", "L4", "C1", "O3", None, "S2", "V1"],
    17: ["T1", "W4", "L2", "C3", "O3", None, "S4", "V3"],
    18: ["T3", "W3", "L3", "C2", "O4", None, "S1", "V2"],
    19: ["T3", "W4", "L4", "C4", "O4", None, "S2", "V4"],
    20: ["T3", "W3", "L1", "C1", "O2", None, "S3", "V4"],
}

# 风格分类集合
tiles = {"T1", "T2", "T3", "T4"}
wallpapers = {"W1", "W2", "W3", "W4"}
lights = {"L1", "L2", "L3", "L4"}
cabinets = {"C1", "C2", "C3", "C4"}
countertops = {"O1", "O2", "O3", "O4"}
sinks = {"S1", "S2", "S3", "S4"}
dishwashers = {"D1", "D2"}
ovens = {"V1", "V2", "V3", "V4"}

all_styles = set.union(tiles, wallpapers, lights, cabinets, countertops, sinks, dishwashers, ovens)

# 仓库风格数量限制
style_limits = {
    "T": 2,
    "W": 2,
    "L": 2,
    "C": 2,
    "O": 3,
    "S": 2,
    "D+V": 4
}

# 建立模型
model = pulp.LpProblem("最大化组合数", pulp.LpMaximize)

# 决策变量
y = {i: pulp.LpVariable(f"y_{i}", cat="Binary") for i in sets}
z = {s: pulp.LpVariable(f"z_{s}", cat="Binary") for s in all_styles}

# 目标函数：尽量多地组出组合
model += pulp.lpSum(y[i] for i in sets)

# 若选组合，其所有零件风格必须在库存中
for i in sets:
    for part in sets[i]:
        if part is not None:
            model += y[i] <= z[part]

# 限制每种风格的数量
model += pulp.lpSum(z[s] for s in tiles)       <= style_limits["T"]     # 瓷砖
model += pulp.lpSum(z[s] for s in wallpapers)  <= style_limits["W"]     # 墙纸
model += pulp.lpSum(z[s] for s in lights)      <= style_limits["L"]     # 灯具
model += pulp.lpSum(z[s] for s in cabinets)    <= style_limits["C"]     # 橱柜
model += pulp.lpSum(z[s] for s in countertops) <= style_limits["O"]     # 台面
model += pulp.lpSum(z[s] for s in sinks)       <= style_limits["S"]     # 水槽
model += pulp.lpSum(z[s] for s in dishwashers.union(ovens)) <= style_limits["D+V"]

# 求解模型
model.solve()

# 输出结果
print("可组装的厨房组合总数：", int(pulp.value(model.objective)))
print("组合编号如下：")
for i in sets:
    if pulp.value(y[i]) == 1:
        print(f"Set {i}: {sets[i]}")

print("\n仓库应存放的风格：")
for s in sorted(all_styles):
    if pulp.value(z[s]) == 1:
        print(s, end=" ")

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/liuliangjie/Library/Python/3.9/lib/python/site-packages/pulp/apis/../solverdir/cbc/osx/i64/cbc /var/folders/vt/qwvfy06n63q184jd56rxd3dh0000gn/T/bb7a4e95d3a34543a2c7edb07856e152-pulp.mps -max -timeMode elapsed -branch -printingOptions all -solution /var/folders/vt/qwvfy06n63q184jd56rxd3dh0000gn/T/bb7a4e95d3a34543a2c7edb07856e152-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 165 COLUMNS
At line 622 RHS
At line 783 BOUNDS
At line 834 ENDATA
Problem MODEL has 160 rows, 50 columns and 336 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 10 - 0.00 seconds
Cgl0004I processed model has 160 rows, 50 columns (50 integer (50 of which binary)) and 336 elements
Cutoff increment increased from 1e-05 to 0.9999
Cbc0038I Initial state - 50 integers unsatisfied sum - 25
Cbc0038I Pass   1: suminf.    

In [2]:
# 版本C：扩展版
# 限制：风格种类数（仓库容量）
style_limits = {
    "T": 2,
    "W": 2,
    "L": 2,
    "C": 2,
    "O": 3,
    "S": 2,
    "V": 3  # 烤箱限制 (c)
}

# 创建模型
model = pulp.LpProblem("最大组合数_版本C", pulp.LpMaximize)

# 决策变量
y = {i: pulp.LpVariable(f"y_{i}", cat="Binary") for i in sets}
z = {s: pulp.LpVariable(f"z_{s}", cat="Binary") for s in all_styles}

# 目标函数
model += pulp.lpSum(y[i] for i in sets)

# 如果选了组合i，其用到的风格必须在仓库中
for i in sets:
    for part in sets[i]:
        if part is not None:
            model += y[i] <= z[part]

# 限制每种风格的数量
model += pulp.lpSum(z[s] for s in tiles)       <= style_limits["T"]     # 瓷砖
model += pulp.lpSum(z[s] for s in wallpapers)  <= style_limits["W"]     # 墙纸
model += pulp.lpSum(z[s] for s in lights)      <= style_limits["L"]     # 灯具
model += pulp.lpSum(z[s] for s in cabinets)    <= style_limits["C"]     # 橱柜
model += pulp.lpSum(z[s] for s in countertops) <= style_limits["O"]     # 台面
model += pulp.lpSum(z[s] for s in sinks)       <= style_limits["S"]     # 水槽
model += pulp.lpSum(z[v] for v in ovens)       <= style_limits["V"]     # 烤箱

# 强制使用所有洗碗机
model += z["D1"] == 1
model += z["D2"] == 1

# 求解模型
model.solve()

# 输出结果
print("可组装的厨房组合总数（扩展后）:", int(pulp.value(model.objective)))
print("组合编号如下：")
for i in sets:
    if pulp.value(y[i]) == 1:
        print(f"Set {i}: {sets[i]}")

print("\n仓库应存放的风格：")
for s in sorted(all_styles):
    if pulp.value(z[s]) == 1:
        print(s, end=" ")

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/liuliangjie/Library/Python/3.9/lib/python/site-packages/pulp/apis/../solverdir/cbc/osx/i64/cbc /var/folders/vt/qwvfy06n63q184jd56rxd3dh0000gn/T/afa58996b53f44c38a4d47123ddf4e32-pulp.mps -max -timeMode elapsed -branch -printingOptions all -solution /var/folders/vt/qwvfy06n63q184jd56rxd3dh0000gn/T/afa58996b53f44c38a4d47123ddf4e32-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 167 COLUMNS
At line 624 RHS
At line 787 BOUNDS
At line 838 ENDATA
Problem MODEL has 162 rows, 50 columns and 336 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 10 - 0.00 seconds
Cgl0004I processed model has 147 rows, 48 columns (48 integer (48 of which binary)) and 308 elements
Cutoff increment increased from 1e-05 to 0.9999
Cbc0038I Initial state - 48 integers unsatisfied sum - 24
Cbc0038I Solution found of 0
C

In [3]:
# 版本D：扩容版
# 限制风格种类（不限制的不写）
style_limits = {
    "T": 2,  # 瓷砖
    "W": 2,  # 墙纸
    "L": 3,  # 灯具
    "C": 3,  # 橱柜
    "V": 3   # 烤箱
}

# 模型
model = pulp.LpProblem("最大组合数_版本D", pulp.LpMaximize)

# 决策变量
y = {i: pulp.LpVariable(f"y_{i}", cat="Binary") for i in sets}
z = {s: pulp.LpVariable(f"z_{s}", cat="Binary") for s in all_styles}

# 目标函数
model += pulp.lpSum(y[i] for i in sets)

# 组合可行性约束：用到的风格必须在仓库中
for i in sets:
    for part in sets[i]:
        if part is not None:
            model += y[i] <= z[part]

# 限制每种风格的数量
model += pulp.lpSum(z[s] for s in tiles)       <= style_limits["T"]     # 瓷砖
model += pulp.lpSum(z[s] for s in wallpapers)  <= style_limits["W"]     # 墙纸
model += pulp.lpSum(z[s] for s in lights)      <= style_limits["L"]     # 灯具
model += pulp.lpSum(z[s] for s in cabinets)    <= style_limits["C"]     # 橱柜
model += pulp.lpSum(z[v] for v in ovens)       <= style_limits["V"]     # 烤箱

# 强制使用所有洗碗机
model += z["D1"] == 1
model += z["D2"] == 1

# 求解
model.solve()

# 输出结果
print("可组装的厨房组合总数（扩容版本）:", int(pulp.value(model.objective)))
print("组合编号如下：")
for i in sets:
    if pulp.value(y[i]) == 1:
        print(f"Set {i}: {sets[i]}")

print("\n仓库应存放的风格：")
for s in sorted(all_styles):
    if pulp.value(z[s]) == 1:
        print(s, end=" ")

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/liuliangjie/Library/Python/3.9/lib/python/site-packages/pulp/apis/../solverdir/cbc/osx/i64/cbc /var/folders/vt/qwvfy06n63q184jd56rxd3dh0000gn/T/d8a1db4459d2494b86400b2c50f24886-pulp.mps -max -timeMode elapsed -branch -printingOptions all -solution /var/folders/vt/qwvfy06n63q184jd56rxd3dh0000gn/T/d8a1db4459d2494b86400b2c50f24886-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 165 COLUMNS
At line 614 RHS
At line 775 BOUNDS
At line 826 ENDATA
Problem MODEL has 160 rows, 50 columns and 328 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 10 - 0.00 seconds
Cgl0004I processed model has 105 rows, 40 columns (40 integer (40 of which binary)) and 220 elements
Cutoff increment increased from 1e-05 to 0.9999
Cbc0038I Initial state - 40 integers unsatisfied sum - 20
Cbc0038I Solution found of 0
C